In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import sys
from pathlib import Path

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

current_cuda_device = None
print(f"is cuda available: {torch.cuda.is_available()}")
print(f"cuda device count: {torch.cuda.device_count()}")
current_cuda_device = torch.cuda.current_device()
print(f"current cuda device: {current_cuda_device}")
print(f"current cuda device name: {torch.cuda.get_device_name(current_cuda_device)}")

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

In [ ]:
modules_path = os.path.abspath("..")
if modules_path not in sys.path:
    sys.path.append(modules_path)

from data.FloodNet import FloodNetDataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean and std
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create train dataset
train_images_dir = Path("../data/FloodNet_dataset/train/image")
train_csv_file = Path("../data/flood_train_rel_paths.csv")
train_data = FloodNetDataset(annotations_file=train_csv_file, img_dir=train_images_dir, transform=transform)

# Create test dataset
test_images_dir = Path("../data/FloodNet_dataset/test/image")
test_csv_file = Path("../data/flood_test_rel_paths.csv")
test_data = FloodNetDataset(annotations_file=test_csv_file, img_dir=test_images_dir, transform=transform)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
for X, y in train_dataloader:
    # print(X)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

In [ ]:
# modyfing ResNet50 for binary classification

model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification

In [ ]:
model = model.to(device)
print(model)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(dataloader: DataLoader, model, loss_fn, optimizer):
    print("Training...")
    size = len(dataloader.dataset)
    current = 0
    model.train()
    # print(f"num batches: {len(dataloader)}")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).float().unsqueeze(1)
        # compute prediction error
        optimizer.zero_grad()

        outputs = model(X)

        loss = loss_fn(outputs, y)

        # backpropagation
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        loss, current = loss.item(), current + len(X)
        print(f"batch: {batch}, loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader: DataLoader, model):
    print("Testing...")
    size = len(dataloader.dataset)
    # print(f"size: {size}")
    num_batches = len(dataloader)
    # print(f"num batches: {num_batches}")
    model.eval()
    correct = 0
    total = 0
    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds.squeeze().long() == y).sum().item()
            total += y.size(0)
    # print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")
    print(f"Test error: \n Accuracy: {correct / total * 100:.2f}%")

In [ ]:
epochs = 10

for epoch in range(epochs):
    print(f"-------------- Epoch {epoch+1} --------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

In [ ]:
from torchinfo import summary
summary(model, input_size=[1, 3, 299, 299])